In [3]:
%pip install -qU \
  langchain==0.1.1 \
  langchain-community==0.0.13 \
  openai==0.27.7 \
  tiktoken==0.4.0 \
  pinecone-client==3.1.0 \

ERROR: Invalid requirement: ''

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [110]:
from langchain_community.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [111]:
loader1 = CSVLoader("../data/output/technical_documents.csv")
doc = loader1.load()
doc

[Document(page_content='topic_name: Market Organization and Structure\nlearning_outcome: explain the main functions of the financial system\nsummary: This reading introduces how the financial system operates and explains how well-functioning financial systems lead to wealthy economies. Financial analysts need to understand how the financial system works because their analyses often lead to trading decisions.  The financial system consists of markets and the financial intermediaries that operate in them. These institutions allow buyers to connect with sellers. They may trade directly with each other when they trade the same instrument or they only may trade indirectly when a financial intermediary connects the buyer to the seller through transactions with each that appear on the intermediary’s balance sheet. The buyer and seller may exchange instruments, cash flows, or risks.  The following points, among others, were made in this reading:  The financial system consists of mechanisms tha

In [119]:
# Assuming `documents` is a list of Document objects
list_of_dictionaries = [
    {
        'page_content': docs.page_content,
        'metadata': {
            'source': docs.metadata['source'],
            'row': docs.metadata['row']
        }
    } for docs in doc
]

list_of_dictionaries

[{'page_content': 'topic_name: Market Organization and Structure\nlearning_outcome: explain the main functions of the financial system\nsummary: This reading introduces how the financial system operates and explains how well-functioning financial systems lead to wealthy economies. Financial analysts need to understand how the financial system works because their analyses often lead to trading decisions.  The financial system consists of markets and the financial intermediaries that operate in them. These institutions allow buyers to connect with sellers. They may trade directly with each other when they trade the same instrument or they only may trade indirectly when a financial intermediary connects the buyer to the seller through transactions with each that appear on the intermediary’s balance sheet. The buyer and seller may exchange instruments, cash flows, or risks.  The following points, among others, were made in this reading:  The financial system consists of mechanisms that all

In [118]:
import tiktoken

tiktoken.encoding_for_model('gpt-3.5-turbo')

<Encoding 'cl100k_base'>

In [26]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

# create the length function


def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)


tiktoken_len("hello I am a chunk of text and using the tiktoken_len function "
             "we can find the length of this chunk of text in tokens")


26

In [113]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["###", "\n", " ", ""]
)

In [126]:
chunks = text_splitter.split_text(list_of_dictionaries[6]['page_content'])[:3]
chunks

['topic_name: Market Organization and Structure\nlearning_outcome: compare execution, validity, and clearing instructions',
 'summary: This reading introduces how the financial system operates and explains how well-functioning financial systems lead to wealthy economies. Financial analysts need to understand how the financial system works because their analyses often lead to trading decisions.  The financial system consists of markets and the financial intermediaries that operate in them. These institutions allow buyers to connect with sellers. They may trade directly with each other when they trade the same instrument or they only may trade indirectly when a financial intermediary connects the buyer to the seller through transactions with each that appear on the intermediary’s balance sheet. The buyer and seller may exchange instruments, cash flows, or risks.  The following points, among others, were made in this reading:  The financial system consists of mechanisms that allow strange

In [127]:
tiktoken_len(chunks[0]), tiktoken_len(chunks[1]), tiktoken_len(chunks[2])

(20, 399, 401)

In [43]:
import os

# get openai api key from platform.openai.com
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') or 'YOUR_API_KEY'

In [44]:
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

/Users/riteshchoudhary/Downloads/Assignment5/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [45]:
texts = [
    'this is the first chunk of text',
    'then another second chunk of text is here'
]

res = embed.embed_documents(texts)
len(res), len(res[0])

(2, 1536)

In [94]:
from json import load
from pinecone import Pinecone
import os
# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.environ.get('PINECONE_API_KEY') or "YOUR_API_KEY"

# configure client
pc = Pinecone(api_key="8db01a1e-c0a4-41b3-8fd9-684a02e57f50")

In [95]:
from pinecone import PodSpec

environment = os.environ.get('PINECONE_ENVIRONMENT')
spec = PodSpec(environment=environment)

In [96]:
index_name = 'assigmment4'

In [97]:
import time

if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

# we create a new index
pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of text-embedding-ada-002
        metric='cosine',
        spec=spec
    )

# wait for index to be initialized
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

In [98]:
index = pc.Index(index_name)
# wait a moment for connection
time.sleep(1)

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [133]:
from langchain.embeddings.openai import OpenAIEmbeddings
from tqdm.auto import tqdm
from uuid import uuid4

batch_limit = 100

texts = []
metadatas = []

for i, record in enumerate(tqdm(list_of_dictionaries)):
    # first get metadata fields for this record
    metadata = {
        'source': record['metadata']['source'],
        'id': record['metadata']['row']
    }
    # now we create chunks from the record text
    record_texts = text_splitter.split_text(
        record['page_content'])
    # create individual metadata dicts for each chunk
    record_metadatas = [{
        "chunk": j, "text": text, **metadata
    } for j, text in enumerate(record_texts)]
    # append these to current batches
    texts.extend(record_texts)
    metadatas.extend(record_metadatas)
    # if we have reached the batch_limit we can add texts
    if len(texts) >= batch_limit:
        ids = [str(uuid4()) for _ in range(len(texts))]
        embeds = embed.embed_documents(texts)
        index.upsert(vectors=zip(ids, embeds, metadatas))
        texts = []
        metadatas = []

if len(texts) > 0:
    ids = [str(uuid4()) for _ in range(len(texts))]
    embeds = embed.embed_documents(texts)
    index.upsert(vectors=zip(ids, embeds, metadatas))

100%|██████████| 31/31 [00:01<00:00, 16.22it/s] 


In [134]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.00148,
 'namespaces': {'': {'vector_count': 148}},
 'total_vector_count': 148}